# GUI - Nine Men Morris

In [ ]:
%%HTML
<style>
.container { width:100% }
</style>

## Definitionen:
- ring = einer der 3 Quadrate (0-2)
- cell = ein Punkt auf einem Ring (0-7)
- position = (ring, cell)
- board = Array aus 3 Arrays, die angeben, ob, und wenn ja welcher Stein dort sitzt
- remaining = \[noch nicht gesetzte Steine Spieler 1, noch nicht gesetzte Steine Spieler 2\]
- pieces = \[remaining, board\]

In [ ]:
%run ./Muehle_Logic.ipynb
%run ./Muehle_Utilities.ipynb
import ipycanvas
from ipycanvas import Canvas, MultiCanvas
import tkinter as tk
from tkinter import ttk

from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

## Konstanten

* BOARD_SIZE = Größe des Spielfeldes in Pixeln
* DOT_RADIUS = Radius der schwarzen kleinen Punkte, die mögliche Positionen markieren (in Abhängigkeit von der Spielfeldgröße)
* PIECE_RADIUS = Radius der Spielsteine
* COLOUR = Farben der [dots, pieces_player_1, pieces_player_2]
* PADDING = relativer Abstand des äüßersten Quadrats zum Spielfeldrand
* DISTANCE = relativer Abstand zwischen den Quadraten des Spielfelds

In [ ]:
BOARD_SIZE      = 400
DOT_RADIUS      = BOARD_SIZE*0.025
PIECE_RADIUS    = BOARD_SIZE*0.04
COLOUR          = ['black', 'white', 'sienna']
COLOUR_HINT     = 'green'
COLOUR_OPPONENT = 'red'
PADDING         = 0.05
DISTANCE        = 0.15
TRANSPARENCY_DEFAULT = 1.0
TRANSPARENCY_HINT    = 0.5

## Mögliche Positionen

<img src="board_positions.png" alt="Mögliche Positionen" width="800"/>

In [ ]:
#Reihe von oben nach unten (0-6)
def row(number):
    return PADDING + DISTANCE * number
#Spalte von links nach rechts (0-6)
def col(number):
    return row(number)

In [ ]:
positions = [([col(0), row(0)], [col(3), row(0)], [col(6), row(0)], [col(6), row(3)], [col(6), row(6)], [col(3), row(6)], [col(0), row(6)], [col(0), row(3)]), #ring 0
             ([col(1), row(1)], [col(3), row(1)], [col(5), row(1)], [col(5), row(3)], [col(5), row(5)], [col(3), row(5)], [col(1), row(5)], [col(1), row(3)]), #ring 1
             ([col(2), row(2)], [col(3), row(2)], [col(4), row(2)], [col(4), row(3)], [col(4), row(4)], [col(3), row(4)], [col(2), row(4)], [col(2), row(3)])] #ring 2

wmd_dic = {
    "a1":positions[0][6],
    "a4":positions[0][7],
    "a7":positions[0][0],
    "b2":positions[1][6],
    "b4":positions[1][7],
    "b6":positions[1][0],
    "c3":positions[2][6],
    "c4":positions[2][7],
    "c5":positions[2][0],
    "d1":positions[0][5],
    "d2":positions[1][5],
    "d3":positions[2][5],
    "d5":positions[2][1],
    "d6":positions[1][1],
    "d7":positions[0][1],
    "e3":positions[2][4],
    "e4":positions[2][3],
    "e5":positions[2][2],
    "f2":positions[1][4],
    "f4":positions[1][3],
    "f6":positions[1][2],
    "g1":positions[0][4],
    "g4":positions[0][3],
    "g7":positions[0][2]
}

## Initialisierung Canvas

**Aufbau Leinwand:**

board\[Hintergrund, Linien, Steine]

In [ ]:
#board[Hintergrund, Linien, Steine]
board = MultiCanvas(3, width = BOARD_SIZE, height = BOARD_SIZE)

# Hintergrund
board[0].fill_style = '#ffffcc'
board[0].fill_rect(0, 0, BOARD_SIZE)

# Strichstärke
board[1].line_width = 5

# Quadrate
board[1].stroke_rect(BOARD_SIZE*col(0), BOARD_SIZE*row(0), BOARD_SIZE*(1-row(0)-col(0))) #ring 0
board[1].stroke_rect(BOARD_SIZE*col(1), BOARD_SIZE*row(1), BOARD_SIZE*(1-row(1)-col(1))) #ring 1
board[1].stroke_rect(BOARD_SIZE*col(2), BOARD_SIZE*row(2), BOARD_SIZE*(1-row(2)-col(2))) #ring 2

# Mittelinien
board[1].begin_path()
board[1].move_to(BOARD_SIZE*col(3), BOARD_SIZE*row(0)) #oben
board[1].line_to(BOARD_SIZE*col(3), BOARD_SIZE*row(2))
board[1].move_to(BOARD_SIZE*col(6), BOARD_SIZE*row(3)) #rechts
board[1].line_to(BOARD_SIZE*col(4), BOARD_SIZE*row(3))
board[1].move_to(BOARD_SIZE*col(3), BOARD_SIZE*row(6)) #unten
board[1].line_to(BOARD_SIZE*col(3), BOARD_SIZE*row(4))
board[1].move_to(BOARD_SIZE*col(0), BOARD_SIZE*row(3)) #links
board[1].line_to(BOARD_SIZE*col(2), BOARD_SIZE*row(3))
board[1].stroke()

# Punkte (außen, mitte, innen)
for ring in positions:
    for x,y in ring:
        board[1].fill_arc(BOARD_SIZE*x, BOARD_SIZE*y, DOT_RADIUS, 0, 360)

## Status des Spielfelds
* 0 = kein Stein
* 1 = weißer Stein
* 2 = brauner Stein

In [ ]:
class status():
    pieces = [[9,9],[          # Anzahl zu setzender Steine (Spieler_1 (Weiß), Spieler_2 (Braun))
            [0,0,0,0,0,0,0,0], # ring 0
            [0,0,0,0,0,0,0,0], # ring 1
            [0,0,0,0,0,0,0,0]  # ring 2
      ]]
    current_player = 1
    mills = set()
    number_stones_to_remove = 0
    selected_dot_old = (None, None)
    selected_dot_new = (None, None)
    move_or_jump = False
    game_over = False

## Initialisierung und Funktionen der Status-Widgets und des Restar-Button

In [ ]:
def on_button_clicked(b):
    status.pieces= [[9,9],[          
                    [0,0,0,0,0,0,0,0], 
                    [0,0,0,0,0,0,0,0], 
                    [0,0,0,0,0,0,0,0]  
                   ]]
    status.current_player = 1
    status.mills = set()
    status.number_stones_to_remove = 0
    status.selected_dot_old = (None, None)
    status.selected_dot_new = (None, None)
    status.move_or_jump = False
    status.game_over = False
    update_board(status)

In [ ]:
turn_label = widgets.Label(value='Spieler x ist an der Reihe')

status_label = widgets.Label(value='aktueller Status:')

pieces_status_label = widgets.Label(value='Setzbare Steine:')

restart_button = widgets.Button(description='restart')
restart_button.on_click(on_button_clicked)

stones_player_one = widgets.FloatProgress(
                value=9,
                max=9,
                min=0,
                description='Spieler 1',
                bar_style= 'info', # 'success', 'info', 'warning', 'danger' or ''
                orientaiton='horizontal'
            )

stones_player_two = widgets.FloatProgress(
                value=9,
                max=9,
                min=0,
                description='Spieler 2',
                bar_style='warning', # 'success', 'info', 'warning', 'danger' or ''
                orientaiton='horizontal'
            )

In [ ]:
def change_status_label(text):
    status_label.value = 'Status: '+text

In [ ]:
def update_status_widgets():
    turn_label.value='Spieler '+ str(status.current_player)+ ' ist dran.'
    status_label.value = 'Status: '+'Spieler in Phase '+ str(get_player_phase(status.pieces, status.current_player))+'.'
    if(status.pieces[0][0] == status.pieces[0][1] == 0):
        pieces_status_label.value = 'Verbleibende Steine:'
        stones_player_one.value = len(player_pieces(status.pieces, 1))
        stones_player_two.value = len(player_pieces(status.pieces, 2))
    else:
        pieces_status_label.value = 'Setzbare Steine:'
        stones_player_one.value = status.pieces[0][0]
        stones_player_two.value = status.pieces[0][1]

## Spielsteine anzeigen

In [ ]:
def draw_piece(ring, cell, player):
    #Hintergrund
    board[2].fill_style = COLOUR[player]
    board[2].fill_arc(BOARD_SIZE*positions[ring][cell][0], BOARD_SIZE*positions[ring][cell][1], PIECE_RADIUS, 0, 360)
    #Ringe
    if player == 1:
        board[2].stroke_style = 'silver' 
    else:
        board[2].stroke_style = 'chocolate'
    board[2].stroke_arc(BOARD_SIZE*positions[ring][cell][0], BOARD_SIZE*positions[ring][cell][1], PIECE_RADIUS,     0, 360)
    board[2].stroke_arc(BOARD_SIZE*positions[ring][cell][0], BOARD_SIZE*positions[ring][cell][1], PIECE_RADIUS*0.7, 0, 360)
    board[2].stroke_arc(BOARD_SIZE*positions[ring][cell][0], BOARD_SIZE*positions[ring][cell][1], PIECE_RADIUS*0.3, 0, 360)

In [ ]:
def update_board(status):
    one_below_other = widgets.VBox([turn_label, status_label, pieces_status_label, stones_player_one, stones_player_two, restart_button])
    side_by_side = widgets.HBox([board, one_below_other])
    display(side_by_side)
    #display(turn_label)
    #display(status_label)
    #display(pieces_status_label)
    #display(stones_player_one)
    #display(stones_player_two)

    with ipycanvas.hold_canvas(board):
        board[2].clear()
        board[2].global_alpha = TRANSPARENCY_DEFAULT
        for ring in range(3):
            for cell in range(8):
                player = status.pieces[1][ring][cell]
                if player in [1, 2]: draw_piece(ring, cell, player)
        board[2].fill_style = 'black'
        board[2].font = '18px serif'
        board[2].fill_text('Spieler '+ str(status.current_player), BOARD_SIZE*0.4, BOARD_SIZE*0.45)
        board[2].fill_text('ist dran.', BOARD_SIZE*0.4, BOARD_SIZE*0.55)  
        update_status_widgets()
    #return board

In [ ]:
def on_dot(pos, x, y):
    return (pos[0]*BOARD_SIZE-PIECE_RADIUS/2 < x < pos[0]*BOARD_SIZE+PIECE_RADIUS/2 and pos[1]*BOARD_SIZE-PIECE_RADIUS/2 < y < pos[1]*BOARD_SIZE+PIECE_RADIUS/2)

In [ ]:
def show_possible_clickable_positions(possible_positions, colour = COLOUR_HINT):
    for position in possible_positions:
        ring, cell = position
        board[2].fill_style = colour
        board[2].global_alpha = TRANSPARENCY_HINT
        board[2].fill_arc(BOARD_SIZE*positions[ring][cell][0], BOARD_SIZE*positions[ring][cell][1], PIECE_RADIUS, 0, 360)
    return

In [ ]:
def winner_popup(winner):
    msg = "Herzlichen Glückwunsch! \nSpieler " + str(winner) + ", du hast gewonnen!"
    popup = tk.Tk()
    popup.wm_title("Gewonnen!")
    label = ttk.Label(popup, text=msg, font=("Verdana", 10))
    label.pack(side="top", fill="x", pady=10)
    B1 = ttk.Button(popup, text="Yeah!", command = popup.destroy)
    B1.pack()
    popup.mainloop()

In [ ]:
def handle_mouse_down(x, y):
    global status
    if not status.game_over:
        for ring in positions:
            for cell in ring:
                if on_dot(cell, x, y):
                    status.selected_dot_old = status.selected_dot_new
                    status.selected_dot_new = (positions.index(ring), ring.index(cell))
                    try:
                        make_move(positions.index(ring), ring.index(cell))
                    except InvalidMoveException:
                        print('Ungültiger Zug')
board[2].on_mouse_down(handle_mouse_down)